# [GSE85217] Tracking outliers - Cook's Distance

In [1]:
!jupyter-lab enable widgetsnbextension

[C 2024-11-01 15:14:32.113 ServerApp] No such file or directory: /home/thomas/Documents/git/medulloblastoma_cavalli_kaggle/code/local_version/nb/enable


## Import lib

In [4]:
# lib
#import modin.pandas as pd
import pandas as pd
import numpy as np
import os
from collections import OrderedDict
import umap

# fig
import matplotlib.pyplot as plt
import seaborn as sns

# local lib
import sys
sys.path.insert(1,'/home/thomas/Documents/git/medulloblastoma_cavalli_kaggle/code/local_version/fun')

from parser import Data

In [5]:
path_data='/home/thomas/Documents/git/medulloblastoma_cavalli_kaggle/data/in/'
path_exp_mat = path_data + 'GSE85217_M_exp_763_MB_SubtypeStudy_TaylorLab_parsed.txt'
path_meta = path_data + 'GSE85217_Cavalli_subgroups_information_parsed.csv'

data=Data()
data.add_exp_mat(path_exp_mat,index_col="genes_name")
data.add_meta(path_meta=path_meta,index_col="samples_name")

In [24]:
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [18]:
def get_design_matrix(metadata,colname,**kwargs):
    return pd.get_dummies(metadata[colname],dtype=int,**kwargs)

In [19]:
get_design_matrix(data.meta,colname="Subtype")

,Group3_alpha,Group3_beta,Group3_gamma,Group4_alpha,Group4_beta,Group4_gamma,SHH_alpha,SHH_beta,SHH_delta,SHH_gamma,WNT_alpha,WNT_beta
samples_name,,,,,,,,,,,,
MB_SubtypeStudy_55001,0,0,0,0,0,0,0,1,0,0,0,0
MB_SubtypeStudy_55002,1,0,0,0,0,0,0,0,0,0,0,0
MB_SubtypeStudy_55003,0,0,0,0,0,1,0,0,0,0,0,0
MB_SubtypeStudy_55004,0,0,0,0,1,0,0,0,0,0,0,0
MB_SubtypeStudy_55005,0,0,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
MB_SubtypeStudy_55759,0,0,0,0,0,0,0,0,1,0,0,0
MB_SubtypeStudy_55760,0,0,0,0,0,0,0,0,1,0,0,0
MB_SubtypeStudy_55761,0,0,0,0,1,0,0,0,0,0,0,0


In [25]:
data.meta["Subtype"].value_counts()

Subtype
Group4_gamma    119
Group4_beta     109
Group4_alpha     98
SHH_delta        76
Group3_alpha     67
SHH_alpha        65
WNT_alpha        49
SHH_gamma        47
Group3_gamma     40
Group3_beta      37
SHH_beta         35
WNT_beta         21
Name: count, dtype: int64

In [34]:
data_nb=pd.concat([data.exp_mat.loc['TSPAN6',data.meta.index.to_list()],data.meta['Subtype']],axis=1)

In [38]:
nb_model=smf.glm('TSPAN6 ~ Subtype',data=data_nb,family=sm.families.NegativeBinomial()).fit()

/home/thomas/anaconda3/lib/python3.10/site-packages/statsmodels/genmod/families/family.py:1367: ValueWarning: Negative binomial dispersion parameter alpha not set. Using default value alpha=1.0.
  warnings.warn("Negative binomial dispersion parameter alpha not "
